### 1. Volatility Risk Model (Price Severity)

**Question it answers**

> *“If the system is stressed, how large could price moves be?”*

This model focuses on **market impact**, not on detecting stress itself.

### What goes into the model
- Recent price behavior (volatility persistence)
- Operational stress indicators
- Weather pressure
- Storage tightness

### What the model produces
- A **distribution of possible price moves**, not a single forecast
- Tail-risk probabilities such as:
  - `P(|price move| > 2%)`
  - `P(|price move| > 3%)`

### Interpretation
- Captures **risk severity**, not direction
- Explicitly models heavy-tailed outcomes (extreme events)

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
import matplotlib.pyplot as plt


In [ ]:
load_dotenv()

In [ ]:
from models.features.build_model_frame import ModelFrameConfig, build_model_frame
from models.features.derive_forecast_inputs import derive_forecast_inputs
from models.volatility.vol_risk_model import fit_vol_risk_model
from models.volatility.forecast_vol_risk import forecast_vol_risk

In [ ]:
MONGO_DB = os.getenv("MONGO_DB")
MONGO_URI = os.getenv("MONGO_URI")

### 1. Build Model Frame

In [ ]:
cfg = ModelFrameConfig(
    pipeline="algonquin",
    capacity_collection="ebb_algonquin_capacity",
    notices_collection="ebb_algonquin_notices",
    start="2025-10-01",
)

In [ ]:
df=build_model_frame(cfg, mongo_uri=MONGO_URI, mongo_db=MONGO_DB)

In [ ]:
# -----------------------------
# Train / Test Split (time-based)
# -----------------------------

train_end = pd.Timestamp("2025-11-01")

df_train = df[df["date"] <= train_end].copy()
df_test = df[df["date"] > train_end].copy()

print(f"Train rows: {len(df_train)}")
print(f"Test rows: {len(df_test)}")
print(df_train["date"].min(), "→", df_train["date"].max())
print(df_test["date"].min(), "→", df_test["date"].max())

### 2. Fit Volatility Risk Model

In [ ]:
model_vol, idata_vol, scalers_vol = fit_vol_risk_model(df_train)

### 4. Out-of-Sample Checks

In [ ]:
results = []

for i in range(len(df_test)):
    row = df_test.iloc[: i + 1]  # expanding window

    x_vol = derive_forecast_inputs(row)

    p_vol = forecast_vol_risk(
        model_vol,
        idata_vol,
        x=x_vol,
        scalers=scalers_vol,
    )[1]

    results.append(
        {
            "date": row.iloc[-1]["date"],
            "p_vol_exceed": p_vol,
        }
    )

df_oos = pd.DataFrame(results)


### 5. Diagnostics & Sanity Checks¶

#### 5.1 Sanity checks

In [ ]:
df_oos.describe()

##### Time series visualization

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(df_oos["date"], df_oos["p_vol_exceed"], label="Volatility Exceedance Prob")
ax.legend()
ax.set_title("Out-of-Sample Risk Signals")
plt.show()

#### 5.2 Volatility risk evaluation (severity model)

##### Define realized volatility exceedance

In [ ]:
VOL_THRESHOLD = 0.02

df_eval["realized_vol_exceed"] = (
    df["hh_ret"].abs() > VOL_THRESHOLD
).astype(int)


##### Compare probabilities

In [ ]:
df_eval.groupby("realized_vol_exceed")["p_vol_exceed"].describe()

##### Reliability check (calibration-lite)

In [ ]:
bins = pd.qcut(df_eval["p_vol_exceed"], q=5)
df_eval.groupby(bins)["realized_vol_exceed"].mean()

#### 5.3 Threshold-based evaluation

In [ ]:
VOL_ALERT = 0.40

df_eval["vol_alert"] = df_eval["p_vol_exceed"] > VOL_ALERT

In [ ]:
pd.crosstab(
    df_eval["vol_alert"],
    df_eval["realized_vol_exceed"],
    normalize="index",
)